In [1]:
import chi
chi.use_site("CHI@UC")
chi.set("project_name", "CHI-210833")

Now using CHI@UC:
URL: https://chi.uc.chameleoncloud.org
Location: Argonne National Laboratory, Lemont, Illinois, USA
Support contact: help@chameleoncloud.org


In [2]:
from chi import lease
reservations = []

node_count = 2
node_type = "gpu_rtx_6000"
lease.add_node_reservation(reservations, count=node_count, node_type=node_type)
lease.add_fip_reservation(reservations, count=node_count)

network_name = "SCC-Net-Example"
lease.add_network_reservation(reservations, network_name)

lease_name = "SCC-Test"
start_date, end_date = lease.lease_duration(days=1)

leased_resources = lease.create_lease(lease_name, reservations, start_date=start_date, end_date=end_date)

In [4]:
from chi import network
lease_id = leased_resources["id"]
lease.wait_for_active(lease_id)
lease_server_id = lease.get_node_reservation(lease_id)

network_id = lease.get_network_id(network_name)

network.create_subnet("scc-subnet-name", network_id)

network.create_router("scc-router-name", gw_network_name=network.PUBLIC_NETWORK)

router_id = network.get_router_id("scc-router-name")
subnet_id = network.get_subnet_id("scc-subnet-name")
network.add_subnet_to_router(router_id, subnet_id)

{'network_id': 'c2ee2244-2efe-4257-a359-af0258df4900',
 'tenant_id': '09f6fb74a74245a6aa1e0d671eb30c95',
 'subnet_id': '1c41ac16-7101-470d-8267-8a7b4912a226',
 'subnet_ids': ['1c41ac16-7101-470d-8267-8a7b4912a226'],
 'port_id': '12733f23-c9b4-4c32-ae89-ed40cfae493c',
 'id': '118533ea-bc00-48a7-aa5d-0328ff10e041'}

In [5]:
from chi import server
server_obj = server.create_server("scc-test-server", 
                                  reservation_id = lease_server_id,
                                  network_name = network_name,
                                  key_name = "msherman-laptop",
                                  image_name="CC-Ubuntu18.04-CUDA10",
                                  count = node_count
                                 )

In [6]:
ip_0 = server.associate_floating_ip(server_obj[0].id)
ip_1 = server.associate_floating_ip(server_obj[1].id)

In [ ]:
print(server.wait_for_active(server_obj[0].id).status)
print(server.wait_for_active(server_obj[1].id).status)

In [ ]:
server.wait_for_tcp(ip_0, 22)
server.wait_for_tcp(ip_1, 22)
